출처 : 혼자 공부하는 머신러닝 + 딥러닝

# **로지스틱 회귀**

In [1]:
# 데이터 불러오기

import pandas as pd
fish = pd.read_csv('http://bit.ly/fish_csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
# 'Species' 열의 고유값 확인

print(pd.unique(fish['Species'])) # 7가지

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [15]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy() # 'Species'를 제외한 열을 numpy 배열로 변환, fish_input에 저장
fish_input.shape # 크기 확인

(159, 5)

In [16]:
print(fish_input[:5]) # fish_input 0, 1, 2, 3, 4 행 출력

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [18]:
fish_target = fish[['Species']].to_numpy() # 'Species' 열을 넘파이 배열로 변환, fish_target에 저장

In [19]:
from sklearn.model_selection import train_test_split # train data와 test data를 나눠주는 함수
train_input, test_input, train_target, test_target = train_test_split(fish_input,
                                                                      fish_target,
                                                                      random_state = 42)
 # random_state를 지정하면 같은 수를 넣었을 때 항상 같은 test data와 train data를 만든다.

In [24]:
from sklearn.preprocessing import StandardScaler # 데이터를 표준화 전처리 하는 함수(범위 : -2 ~ +2)
ss = StandardScaler()
ss.fit(train_input) # train_input data 표준화 모델 만들기
train_scaled = ss.transform(train_input) # train_input 표준화
test_scaled = ss.transform(test_input) # test_input 표준화

# k-최근접 이웃 분류기(KNN)

In [26]:
from sklearn.neighbors import KNeighborsClassifier # k-최근접 이웃 분류 함수
kn = KNeighborsClassifier(n_neighbors=3) # 'n_neighbors' 파라미터는 확인할 최근접 데이터의 수를 지정
kn.fit(train_scaled, train_target) # KNN 모델링
print(kn.score(train_scaled, train_target)) # 학습된 모델로 train data 예측시 점수
print(kn.score(test_scaled, test_target)) # 학습된 모델로 test data 예측시 점수

# train set을 예측했으때 점수가 더 높은 것을 봤을 때 train data에 과적합된 경향이 있다.

0.8907563025210085
0.85


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
